# COSADAMA Introduction to Data Science Study

# WEEK3 : [프로젝트] 서울시 범죄 현황 분석

- 작성자: 송혜민
- 참고자료: 2020/2021 코사다마 데이터 분석 커리큘럼(박하람), 파이썬 머신러닝 완벽가이드(권철민), 비쥬얼프로그래밍 수업(김승태), 파이썬으로 데이터 주무르기(민형기)

이번 주에는 서울시 범죄 현황 분석 프로젝트를 통해 이전 주차에 배웠던 pandas를 더 배워보는 시간을 가질 예정입니다. 뿐만 아니라, Google Maps를 활용하여 주소 및 경/위도를 찾아보고, 새로운 시각화 도구인 Seaborn과 Folium을 통한 지도 시각화까지 배우게 됩니다. 또한, sklearn의 preprocessing 모듈을 활용하여 정규화(normalize)까지 해보려고 합니다. 

이전에 배웠던 것들을 복습하는 것과 더불어 새롭게 배우는 내용들에 관해 꼼꼼하게 확인하시면서 따라와주시는게 좋겠죠😊


### 1. 문제 정의하기 

구체적으로 **서울시 범죄 현황 분석**을 해보려고 합니다. [2014년 신문 기사](https://www.asiae.co.kr/article/2014102008312459233)에 따르면 강남 3구의 체감안전도가 높은 것으로 파악되었다고 합니다. 또 다른 [신문 기사](https://news.mt.co.kr/mtview.php?no=2021011215094899686)에 따르면 2021년 서울시 사회안전지수는 용산구, 강남구, 강동구, 서초구, 마포구, 송파구 순으로 높게 평가되었습니다. 2021년 기준으로도 2014년과 마찬가지로 안전에 있어 강남 3구의 안전 수치가 일정부분 높게 파악되었다고 할 수 있습니다. 따라서 이번 프로젝트는 **강남 3구의 (체감)안전도가 높다는 기사를 검증**해보려 합니다. '서울시 구별 범죄 발생과 검거율'을 활용하여 검증할 예정이구요. 동시에 지난 주에 사용했던 인구 데이터도 함께 활용하여 인구별 범죄현황까지 알아볼 겁니다. 


### 2. 데이터 수집하기

이번 프로젝트에서는 공공데이터포털의 데이터를 활용할 예정입니다. 
- [공공데이터포털 - 서울특별시 관서별 5대 범죄 현황](https://www.data.go.kr/data/15054738/fileData.do) 

목적이 사회안전지수가 높은 구가 실제로 안전한지를 검증하려는 것이죠? 우리는 이미 활용할 데이터가 주어져 있기 때문에 이런 걱정을 하지 않아도 되지만 목적에 적합하도록 **어떤 데이터를 수집할 것인가**를 결정하는 것은 매우 중요합니다! 안전을 증명할 수 있으려면 어떤 데이터를 모아야 할까요? 여러 고민을 해보시면서 공공데이터포털 및 다양한 사이트에서 맞는 데이터를 검색해보시길 바랍니다. 


### 3. pandas를 이용하여 데이터 정리하기

In [ ]:
df = pd.read_csv("C:\\Users\\user\\Downloads\\경찰청 서울특별시경찰청_경찰서별 5대범죄 발생 검거 현황_12_31_2020.csv", 
                 encoding='euc-kr')
df.head()

,구분,죄종,발생검거,건수
0,중부,살인,발생,1
1,중부,살인,검거,1
2,중부,강도,발생,3
3,중부,강도,검거,4
4,중부,"강간,추행",발생,113


In [ ]:
df['범죄 검거'] = df["죄종"].map(str) + " " + df["발생검거"] #죄종, 발생검거 열의 텍스트를 결합
df = df.drop(['죄종', '발생검거'], axis = 1)
df.head()

,구분,건수,범죄 검거
0,중부,1,살인 발생
1,중부,1,살인 검거
2,중부,3,강도 발생
3,중부,4,강도 검거
4,중부,113,"강간,추행 발생"


In [ ]:
crimes = df['범죄 검거'].unique()

data = []
for n in range(11) : 
        data.append(n)

crime_anal_police = df[df['범죄 검거'] == crimes[0]]
crime_anal_police = crime_anal_police.drop(['범죄 검거'], axis = 1)
crime_anal_police.columns = ['관서명', crimes[0]]
crime_anal_police = crime_anal_police.reset_index(drop = True)

for n in range(1, 10) : 
        data[n] = df[df['범죄 검거'] == crimes[n]]
        data[n] = data[n].drop(['범죄 검거'], axis = 1)
        data[n].columns = ['관서명', crimes[n]]
        data[n] = data[n].reset_index(drop = True)
        crime_anal_police = pd.merge(crime_anal_police, data[n], how = 'outer')

crime_anal_police = crime_anal_police.fillna(0) #nan값을 0으로 대체
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,"강간,추행 발생","강간,추행 검거",절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부,1,1,3,4,113.0,76.0,943,495,997,830
1,종로,9,6,4,4,115.0,93.0,602,284,964,792
2,남대문,1,1,2,2,73.0,52.0,699,368,579,494
3,서대문,3,3,2,1,159.0,149.0,1077,524,1292,1029
4,혜화,0,0,3,3,77.0,53.0,581,310,747,628


자, 이제 본격적인 분석을 위한 데이터 정리가 완료되었습니다! 

그리고, 분석 전에 데이터 타입이 어떻게 되어있는지 확인해야 할 필요가 있겠죠?

In [ ]:
crime_anal_police.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 0 to 30
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   관서명       31 non-null     object 
 1   살인 발생     31 non-null     int64  
 2   살인 검거     31 non-null     int64  
 3   강도 발생     31 non-null     int64  
 4   강도 검거     31 non-null     int64  
 5   강간,추행 발생  31 non-null     float64
 6   강간,추행 검거  31 non-null     float64
 7   절도 발생     31 non-null     int64  
 8   절도 검거     31 non-null     int64  
 9   폭력 발생     31 non-null     int64  
 10  폭력 검거     31 non-null     int64  
dtypes: float64(2), int64(8), object(1)
memory usage: 2.9+ KB


관서명을 제외한 모든 데이터가 int64와 float64로 나타나고 있네요. 따로 데이터 타입을 바꿀 필요 없이 그대로 분석을 진행하면 되겠습니다.

자, 그럼 다시 한번 우리가 분석해볼 데이터를 보죠.

In [ ]:
crime_anal_police.head(20)

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,"강간,추행 발생","강간,추행 검거",절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부,1,1,3,4,113.0,76.0,943,495,997,830
1,종로,9,6,4,4,115.0,93.0,602,284,964,792
2,남대문,1,1,2,2,73.0,52.0,699,368,579,494
3,서대문,3,3,2,1,159.0,149.0,1077,524,1292,1029
4,혜화,0,0,3,3,77.0,53.0,581,310,747,628
5,용산,5,3,6,6,230.0,186.0,1111,534,1617,1327
6,성북,0,0,2,2,0.0,0.0,439,233,672,610
7,동대문,10,8,6,5,161.0,142.0,1440,713,1784,1597
8,마포,9,8,4,3,348.0,287.0,1483,679,1844,1557
9,영등포,7,4,6,5,315.0,252.0,2188,953,2701,2157


경찰서 이름이 관서명으로 되어 있습니다. 우리는 이후 서울특별시 구별에 따라 시각화하여 표현하고 싶기 때문에 이를 관서명이 아닌, '구별'로 바꿔주고 싶어요. 같은 구에 여러 관서가 있을 수 있기 때문에, 주소를 정확히 파악해서 같은 구에 있는 것은 합춰주어 하나의 구로 표시해보겠습니다. 이를 위해서 우리는 Google Maps를 사용할 거에요! 
